In [1]:
from test_casadi import *
import casadi as cs

In [2]:
x = cs.SX.sym("x", 4)

In [3]:
f = x[0]*x[3]*cs.sum1(x[:3]) + x[2]
geq = cs.sumsqr(x)
gineq = x[0]*x[1]*x[2]*x[3]
g = cs.vertcat(geq, gineq)
nlp = {'x':x, 'f':f, 'g':g}

In [4]:
opts=dict(
    qpsol="qpoases",
    qpsol_options={"printLevel":"none"}
)
solver = cs.nlpsol("solver", "sqpmethod", nlp, opts)


qpOASES -- An Implementation of the Online Active Set Strategy.
Copyright (C) 2007-2015 by Hans Joachim Ferreau, Andreas Potschka,
Christian Kirches et al. All rights reserved.

qpOASES is distributed under the terms of the 
GNU Lesser General Public License 2.1 in the hope that it will be 
useful, but WITHOUT ANY WARRANTY; without even the implied warranty 
of MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. 
See the GNU Lesser General Public License for more details.

-------------------------------------------
This is casadi::Sqpmethod.
Using exact Hessian
Number of variables:                               4
Number of constraints:                             2
Number of nonzeros in constraint Jacobian:         8
Number of nonzeros in Lagrangian Hessian:         16



In [7]:
x0 = [1., 5., 5., 1.]
lbx, ubx = 1, 5
lbg = [40., 25.]
ubg = [40., cs.inf]
sol = solver(x0=x0, lbx=lbx, ubx=ubx, lbg=lbg, ubg=ubg)

iter      objective    inf_pr    inf_du     ||d||  lg(rg) ls    info
   0   1.600000e+01  1.20e+01  1.20e+01  0.00e+00       -  0  - 
WARNING(sqpmethod): Indefinite Hessian detected
   1   1.625000e+01  1.62e+00  2.81e+00  1.25e+00       -  1  - 
WARNING(sqpmethod): Indefinite Hessian detected
   2   1.622379e+01  1.38e+00  5.06e-01  2.86e-01       -  1  - 
   3   1.700011e+01  2.22e-02  4.16e-02  9.27e-02       -  1  - 
   4   1.701398e+01  7.28e-05  4.22e-05  6.09e-03       -  1  - 
   5   1.701402e+01  1.91e-09  8.46e-10  3.58e-05       -  1  - 
MESSAGE(sqpmethod): Convergence achieved after 5 iterations
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
          QP  | 117.00us ( 23.40us) 111.60us ( 22.32us)         5
  linesearch  |  16.00us (  3.20us)  14.71us (  2.94us)         5
      nlp_fg  |  18.00us (  1.20us)  16.11us (  1.07us)        15
  nlp_hess_l  |  20.00us (  1.33us)  18.07us (  1.20us)        15
  nlp_jac_fg  |  25.00us (  1.39us)  25.99us (  1.44us)

In [8]:
#f = x[0]*x[3]*cs.sum1(x[:3]) + x[2]
x = cs.MX.sym('x', 4, 1)
fn = jax.jit(lambda x: x[0]*x[3]*jnp.sum(x[:3]) + x[2])
f = JaxFn(4, 1, fn)
geq = cs.sumsqr(x)
gineq = x[0]*x[1]*x[2]*x[3]
g = cs.vertcat(geq, gineq)

nlp = {'x':x, 'f':f(x), 'g':g}

In [10]:
opts=dict(qpsol="qpoases",
          qpsol_options={"printLevel":"none"},
          hessian_approximation="limited-memory")
solver = cs.nlpsol("solver", "sqpmethod", nlp, opts)


qpOASES -- An Implementation of the Online Active Set Strategy.
Copyright (C) 2007-2015 by Hans Joachim Ferreau, Andreas Potschka,
Christian Kirches et al. All rights reserved.

qpOASES is distributed under the terms of the 
GNU Lesser General Public License 2.1 in the hope that it will be 
useful, but WITHOUT ANY WARRANTY; without even the implied warranty 
of MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. 
See the GNU Lesser General Public License for more details.

-------------------------------------------
This is casadi::Sqpmethod.
Using limited memory BFGS Hessian approximation
Number of variables:                               4
Number of constraints:                             2
Number of nonzeros in constraint Jacobian:         8
Number of nonzeros in Lagrangian Hessian:         16



In [11]:
x0 = [1., 5., 5., 1.]
lbx, ubx = 1, 5
lbg = [40., 25.]
ubg = [40., cs.inf]
sol = solver(x0=x0, lbx=lbx, ubx=ubx, lbg=lbg, ubg=ubg)

TypeError: Cannot apply JAX transformations to a function lowered and compiled for a particular signature. Detected argument of Tracer type <class 'jax._src.interpreters.partial_eval.DynamicJaxprTracer'>.

Function JaxFn (0x3f2772e0)
Input 0 (i0): [1, 5, 5, 1]
Function nlp_jac_fg (0x2f36660)
Input 0 (x): [1, 5, 5, 1]
Input 1 (p): NULL


RuntimeError: Error in Function::call for 'solver' [Sqpmethod] at .../casadi/core/function.cpp:1401:
Error in Function::call for 'solver' [Sqpmethod] at .../casadi/core/function.cpp:330:
.../casadi/core/oracle_function.cpp:285: Error in solver:nlp_jac_fg:Error in Function::operator() for 'nlp_jac_fg' [MXFunction] at .../casadi/core/function.cpp:1482:
Error in Function::operator() for 'JaxFn' [CallbackInternal] at .../casadi/core/function.cpp:1482:
.../casadi/core/function_internal.cpp:3629: Failed to evaluate 'eval_dm' for JaxFn:
.../casadi/core/callback_internal.cpp:116: Error calling "eval" for object JaxFn:
/work/swig/python/target3/source/casadiPYTHON_wrap.cxx:3810: Cannot apply JAX transformations to a function lowered and compiled for a particular signature. Detected argument of Tracer type <class 'jax._src.interpreters.partial_eval.DynamicJaxprTracer'>.

In [90]:
sol

{'f': DM(17.014),
 'g': DM([40, 25]),
 'lam_g': DM([0.161469, -0.552294]),
 'lam_p': DM(0x0),
 'lam_x': DM([-1.08787, 0, 0, 0]),
 'x': DM([1, 4.743, 3.82115, 1.37941])}

In [63]:
class Fn(cs.Callback):
    def __init__(self, name, fn, opts={}):
        self.fn = fn
        cs.Callback.__init__(self)
        self.construct(name, opts)

    def eval(self, arg):
        x = jnp.asarray(arg[0]).flatten()
        y = cs.vertcat(self.fn(x))
        return [y]

In [67]:
f = Fn('obj', fn, {'enable_fd':True})

In [69]:
x = cs.MX.sym("x")
solver = cs.nlpsol("solver","ipopt",{"x":x,"f":f(x)})
res = solver(x0=x0)

RuntimeError: Error in Function::call for 'solver' [IpoptInterface] at .../casadi/core/function.cpp:1401:
Error in Function::call for 'solver' [IpoptInterface] at .../casadi/core/function.cpp:330:
.../casadi/core/function_internal.hpp:1644: Input 0 (x0) has mismatching shape. Got 4-by-1. Allowed dimensions, in general, are:
 - The input dimension N-by-M (here 1-by-1)
 - A scalar, i.e. 1-by-1
 - M-by-N if N=1 or M=1 (i.e. a transposed vector)
 - N-by-M1 if K*M1=M for some K (argument repeated horizontally)
 - N-by-P*M, indicating evaluation with multiple arguments (P must be a multiple of 1 for consistency with previous inputs)

In [2]:


x0 = cs.DM([1., 5., 5., 1.])
x = cs.MX.sym("x",4)
f = JaxFn((4,1), (1,1), fn)
J = cs.Function('J',[x],[cs.jacobian(f(x),x)])

In [10]:
J(x0) @ x0

DM(38)

In [5]:
jax.jacfwd(fn)(jnp.array([1.,5,5,1]))

Array([12.,  1.,  2., 11.], dtype=float32)